Read `about_the_implementation.md` for more info.

# Prior

In [1]:
import numpy as np
from base_dist import *

In [2]:
# hyperparameters
n, k, p = 40, 3, 4
gamma_0 = np.array([0.6,0.6,0.6,1, 0, 3, 1.1, 2.2, 0.1, -0.3])
V_0 = np.eye(len(gamma_0))
e_0 = 3 * np.ones(k)

# prior
theta_prior = define_theta_prior(gamma_0, V_0, e_0)
prior = define_prior(theta_prior, n)

# test
particle = prior.rvs()

# Dataset

In [3]:
import os
os.chdir('../generate_from_model/')
from generate_dataset import *
os.chdir('../strategy3/')

In [4]:
# generate dataset
alpha, beta, nu = generate_network_params(k, gamma_0, V_0, e_0)
X = generate_covariates(n, p)
sample_from_network((alpha, beta, nu), X)
theta = (alpha,beta,nu)
Y = sample_from_network(theta,X, return_Z=False)

# Logtarget

In [5]:
# to be moved in a script

from scipy.stats import poisson

def compute_llh_marginal(particle, X, Y, k):
    """Vectorized."""
    # extract
    Z_categorical, theta = particle["Z"], particle["theta"]
    gamma, nu = theta["gamma"], theta["nu"]
    alpha,beta = gamma_to_alpha_beta(gamma, k)
    n = len(X)
    p = X.shape[1]
    Z = np.zeros((n, k))
    Z[np.arange(n), Z_categorical] = 1

    # compute
    logpz = np.sum(np.log(Z.dot(nu)))
    assert not np.any(np.isinf(logpz)) and not np.any(np.isnan(logpz))
    lambda_poisson = np.exp(Z @ alpha @ Z.T + X.dot(beta))
    indices = np.triu_indices(n, k=1)
    logpy = np.sum(poisson.logpmf(Y[indices], lambda_poisson[indices]))
    return logpz + logpy

def define_llh_target(prior, X, Y, k):

    def compute_llh_target(particle):
        """Y and X to be defined globally."""
        return compute_llh_marginal(particle, X, Y, k) + prior.logpdf(particle)
    
    return compute_llh_target

In [6]:
compute_llh_target = define_llh_target(prior, X, Y, k)

In [7]:
from particles.smc_samplers import TemperingBridge

class ToyBridge(TemperingBridge):
    def logtarget(self, theta):
        return compute_llh_target(theta)

In [8]:
toy_bridge = ToyBridge(base_dist=prior)

In [9]:
prior.rvs()["theta"]["nu"]

array([[0.49107849, 0.4309189 , 0.07800261]])

# Move

In [10]:
# no base class for a MCMC step

class MCMCSequence:
    """Base class for a (fixed length or adaptive) sequence of MCMC steps."""

    def __init__(self, mcmc=None, len_chain=10):
        self.mcmc = ArrayRandomWalk() if mcmc is None else mcmc
        self.nsteps = len_chain - 1

    def calibrate(self, W, x):
        self.mcmc.calibrate(W, x)

    def __call__(self, x, target):
        for _ in range(self.nsteps):
            x = self.mcmc(x, target)
        return x

In [11]:
from scipy.stats import multivariate_normal

def sample_nu(particles, e_0, e_tilde, rho, k): 
    """Sample nu.
    All particles at once."""
    Z_categorical = particles["Z"]
    assert Z_categorical.ndim == 2
    N,n = Z_categorical.shape
    Z = np.zeros((N, n, k))
    Z[np.arange(N),np.arange(n), Z_categorical] = 1
    e = rho*(e_0+np.sum(Z, axis=1))+(1-rho)*e_tilde
    print("e: ",e.shape)
    nu = dirichlet.rvs(e,size=N) # accept vectorized e ?
    return nu

def compute_gamma_var(particles):
    """Compute the variance of gamma.
    All particles at once."""
    gamma = particles["theta"]["gamma"]
    empirical_gamma_var = gamma.T@gamma/len(gamma)
    return empirical_gamma_var

def sample_gamma(particles, gamma_var, target):
    """Sample gamma.
    Particle by particle proposal.
    To be vectorized or compiled."""
    gamma = particles["theta"]["gamma"]
    N, len_gamma = gamma.shape
    gamma_step = np.random.multivariate_normal.rvs(mean=np.zeros(len_gamma), cov=gamma_var, size=N)

    # particle by particle
    for i in range(N):
        # gamma proposal
        gamma_proposal = gamma[i] + gamma_step[i]
        particle_proposal = particles[i].copy()
        particle_proposal["theta"]["gamma"] = gamma_proposal

        # accept/reject
        logratio = target.logpdf(particle_proposal) - target.logpdf(particles[i])
        ratio = np.exp(logratio)
        if np.random.rand() < ratio:
            particles[i] = particle_proposal
    
    return particles

def sample_Z(particles, target, k):
    """Sample Z.
    Particle by particle proposal.
    To be vectorized or compiled."""
    N = len(particles)

    # particle by particle
    for i in range(N):
        # compute Z probabilities
        Z_logprobas = np.zeros(k)
        for idx in range(k):
            particle_proposal = particles[i].copy()
            particle_proposal["Z"] = idx
            Z_probas[idx] = target.logpdf(particle_proposal) - target.logpdf(particles[i])
        Z_probas = np.exp(Z_logprobas-np.max(Z_logprobas))
        Z_probas /= np.sum(Z_probas)
        # sample Z
        particles["Z"][i] = np.random.choice(np.arange(k), p=Z_probas)
    
    return particles

class CustomGibbs():
    """A custom Gibbs step of the kernel.
    Beware, in that case, x is a ThetaParticles object."""

    def __init__(self, k, e_0, e_tilde):
        self.k = k
        self.e_0 = e_0
        self.e_tilde = e_tilde

    def calibrate(self, W, x):
        self.rho = x.shared["exponents"][-1]
        self.gamma_var = compute_gamma_var(x)
        raise NotImplementedError

    def __call__(self, x, target):
        x["theta"]["nu"] = sample_nu(x, self.e_0, self.e_tilde, self.rho, self.k)
        x["theta"]["gamma"] = sample_gamma(x, self.gamma_var, target)
        x["Z"] = sample_Z(x, target, self.k)
        raise x

In [12]:
move = MCMCSequence(mcmc=CustomGibbs(k, e_0, e_0), len_chain=5)

# AdaptiveTempering

In [13]:
from particles.smc_samplers import AdaptiveTempering
import particles

# finally
fk_tpr = AdaptiveTempering(model=toy_bridge, len_chain=1, move=move)
alg = particles.SMC(fk=fk_tpr, N=200)
alg.run()

TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'

# Debug

In [ ]:
from particles.smc_samplers import MCMCSequenceWF, AdaptiveTempering, ArrayRandomWalk
import particles

from particles.smc_samplers import ThetaParticles
x0 = ThetaParticles(theta=alg.fk.model.prior.rvs(size=100))

In [ ]:
x0.theta["Z"][0]

array([1., 1., 1., 1., 2., 0., 1., 0., 2., 0., 2., 0., 1., 0., 2., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 2., 0., 1., 2., 1.,
       0., 1., 1., 1., 2., 1.])

In [ ]:
alg.fk